In [13]:
import tkinter as tk
from tkinter import messagebox
import random
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import re

# Global Variables
email_attempts = 0
otp_attempts = 0
max_attempts = 3
otp = None

# Utility Functions
def generate_otp():
    try:
        return random.randint(100000, 999999)
    except Exception as e:
        print(f"Error generating OTP: {e}")
        return None

def send_otp_via_email(otp, sender_email_address, recepient_email_address):
    try:
        msg = MIMEMultipart()
        msg['Subject'] = 'Your OTP Code'
        msg['From'] = sender_email_address
        msg['To'] = recepient_email_address
        msg.attach(MIMEText(f"Your OTP is {otp}.", 'plain'))

        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
            password = 'zdou bbvx shep mhrm' # Replace with your actual email password
            server.login(sender_email_address, password)
            server.sendmail(msg['From'], [msg['To']], msg.as_string())
        print("OTP sent successfully.")
        return True
    except smtplib.SMTPException as smtp_error:
        print(f"SMTP error occurred: {smtp_error}")
        return False
    except Exception as e:
        print(f"Failed to send OTP: {e}")
        return False

def validate_email(email):
    try:
        if email.count('@') != 1:
            raise ValueError("Email must contain exactly one '@' symbol.")
        local_part, domain_part = email.split('@')
        if not local_part or local_part.startswith('.') or local_part.endswith('.') or '..' in local_part:
            raise ValueError("Invalid local part of the email.")
        if not re.match(r'^[a-zA-Z0-9_.+-]+$', local_part):
            raise ValueError("Local part contains invalid characters.")
        if not domain_part or domain_part.startswith('-') or domain_part.endswith('-') or '..' in domain_part:
            raise ValueError("Invalid domain part of the email.")
        if not re.match(r'^[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$', domain_part):
            raise ValueError("Invalid domain part of the email.")
        if len(email) > 254:
            raise ValueError("Email address exceeds the maximum length of 254 characters.")
        return True
    except ValueError as ve:
        print(f"Validation Error: {ve}")
        return False
    except Exception as e:
        print(f"Unexpected Error: {e}")
        return False

def verify_otp(generated_otp, entered_otp):
    return generated_otp == entered_otp

def get_valid_email(email_entry, root):
    global email_attempts
    email_attempts += 1

    email = email_entry.get()
    if validate_email(email):
        return email
    else:
        messagebox.showerror("Invalid Email", f"Please enter a valid email address. Attempts remaining: {max_attempts - email_attempts}")
        if email_attempts >= max_attempts:
            messagebox.showerror("Error", "Maximum email attempts reached. Exiting the application.")
            root.destroy()
        return None

def prompt_user_for_otp(otp_entry, root):
    global otp_attempts
    otp_attempts += 1

    entered_otp = otp_entry.get()
    if len(entered_otp) != 6 or not entered_otp.isdigit():
        messagebox.showerror("Invalid OTP", f"Please enter a valid 6-digit OTP. Attempts remaining: {max_attempts - otp_attempts}")
        if otp_attempts >= max_attempts:
            messagebox.showerror("Error", "Maximum OTP attempts reached. Exiting the application.")
            root.destroy()
        return None
    return int(entered_otp)

def handle_send_otp(email_entry, otp_entry, send_otp_button, submit_otp_button, root):
    global otp

    recepient_email_address = get_valid_email(email_entry, root)
    if recepient_email_address is None:
        return

    otp = generate_otp()
    if otp is None:
        messagebox.showerror("Error", "Failed to generate OTP. Exiting.")
        root.destroy()
        return

    sender_email_address = 'preethambp7@gmail.com'
    is_otp_sent = send_otp_via_email(otp, sender_email_address, recepient_email_address)
    if is_otp_sent:
        otp_entry.config(state="normal")
        submit_otp_button.config(state="normal")
        send_otp_button.config(state="disabled")
        messagebox.showinfo("OTP Sent", "OTP has been sent to your email. Please enter it below.")
    else:
        messagebox.showerror("Error", "Failed to send OTP. Please try again.")
        root.destroy()

def handle_verify_otp(otp_entry, root):
    global otp

    entered_otp = prompt_user_for_otp(otp_entry, root)
    if entered_otp is None:
        return

    if verify_otp(otp, entered_otp):
        messagebox.showinfo("Success", "Access granted.")
        root.destroy()
    else:
        if otp_attempts >= max_attempts:
            messagebox.showerror("Error", "Maximum OTP attempts reached. Exiting the application.")
            root.destroy()
        else:
            messagebox.showerror("Failure", f"Incorrect OTP. Attempts remaining: {max_attempts - otp_attempts}")


In [14]:
def main():
    global otp_entry, email_entry, send_otp_button, submit_otp_button, root

    # UI setup
    root = tk.Tk()
    root.title("OTP Authentication")

    tk.Label(root, text="Enter Recipient Email:").grid(row=0, column=0, padx=10, pady=10)
    email_entry = tk.Entry(root, width=30)
    email_entry.grid(row=0, column=1, padx=10, pady=10)

    send_otp_button = tk.Button(root, text="Send OTP", command=lambda: handle_send_otp(email_entry, otp_entry, send_otp_button, submit_otp_button, root))
    send_otp_button.grid(row=0, column=2, padx=10, pady=10)

    tk.Label(root, text="Enter OTP:").grid(row=1, column=0, padx=10, pady=10)
    otp_entry = tk.Entry(root, width=30, state="disabled")
    otp_entry.grid(row=1, column=1, padx=10, pady=10)

    submit_otp_button = tk.Button(root, text="Submit OTP", command=lambda: handle_verify_otp(otp_entry, root), state="disabled")
    submit_otp_button.grid(row=1, column=2, padx=10, pady=10)

    root.mainloop()

if __name__ == "__main__":
    main()

OTP sent successfully.
